# Carson Slater - MATH 493
## EDA and Data Cleaning for SKU Data
### Importing Packages

In [2]:
import pandas as pd
import numpy as np

### Objectives
 - Clean data in order that it is ready for EDA
 - Make plots and conduct numerical summaries of the data

### Import the Data

In [3]:
# If you are not using this on Carson Slater's local device, please change the pathname to locate the data on your local device
baskets = pd.read_csv('/Users/carson/Documents/Wheaton Senior Year/Fall 2022/Mentored Research/baskets_sample_random_10.csv')

### Check for Missing Data

In [4]:
baskets.isna().sum()

id             0
order_id       0
placed_at      0
merchant_id    0
sku_id         0
top_cat_id     7
sub_cat_id     7
qty            0
price          0
dtype: int64

 - There exists seven data missing the `top_cat_id` and `sub_cat_id` variables.
 
 ### Seeing the Missing Data (Since there is so little)

In [5]:
baskets[baskets['top_cat_id'].isnull()]

,id,order_id,placed_at,merchant_id,sku_id,top_cat_id,sub_cat_id,qty,price
51,1129,239,2021-05-20 08:39:59.757,168,553,NaN,NaN,1,43000.0
14844,160923,26349,2021-12-30 14:28:26.393,1717,61,NaN,NaN,2,0.0
24669,276037,50151,2022-04-04 12:49:14.586,1717,1874,NaN,NaN,1,75000.0
28517,329430,58639,2022-07-09 10:00:12.363,629,830,NaN,NaN,4,218000.0
28518,329431,58639,2022-07-09 10:00:12.363,629,2329,NaN,NaN,4,182500.0
28721,331470,59859,2022-07-13 22:40:39.552,185,2382,NaN,NaN,1,97500.0
29149,336523,59976,2022-07-25 13:24:41.172,629,830,NaN,NaN,2,218000.0


### What Kind of Missing Data?

 - To explore the three types of missing data, check out the [NIH website](https://www.ncbi.nlm.nih.gov/books/NBK493614/).
 - Because five of the seven transactions with missing data are from the same two merchants, it is difficult to make the case that these data are missing completely at random (MCAR). The case can be made for missing at random (MAR), because all but two of these missing data are from the same order, as indicated by the `placed_by` variable. Therefore, we will assume our data is missing at random, which is less robust, but the small amount of missing data will mean our analyses will be more robust than most analyses done with MAR data.
 
### Finding and Removing Duplicates

In [6]:
baskets[baskets.duplicated(keep=False)]

,id,order_id,placed_at,merchant_id,sku_id,top_cat_id,sub_cat_id,qty,price
20713,228956,40832,2022-02-21 15:39:38.950,1419,835,12.0,1.0,15,18500.0
20714,228956,40832,2022-02-21 15:39:38.950,1419,835,12.0,1.0,15,18500.0
20715,228958,40832,2022-02-21 15:39:38.950,1419,836,15.0,1.0,5,29500.0
20716,228958,40832,2022-02-21 15:39:38.950,1419,836,15.0,1.0,5,29500.0


 - Above is the duplicated data. It appears to have been a system glitch in the transaction log or perhaps a manual entry error.

In [7]:
# Drop the duplicates
baskets = baskets.drop_duplicates(keep=False)

### Converting Dates and Times

In [8]:
from datetime import datetime, timedelta
baskets['datetime'] = pd.to_datetime(baskets['placed_at'])

baskets['date'] = baskets['datetime'].apply(lambda x: datetime.date(x))
baskets['year'] = baskets['datetime'].apply(lambda x: x.year)
baskets['month'] = baskets['datetime'].apply(lambda x: x.month)
baskets['day'] = baskets['datetime'].apply(lambda x: x.day)
baskets['hour'] = baskets['datetime'].apply(lambda x: x.hour)
baskets['weekday'] = baskets['datetime'].apply(lambda x: datetime.isoweekday(x))
baskets.head(3)

,id,order_id,placed_at,merchant_id,sku_id,top_cat_id,sub_cat_id,qty,price,datetime,date,year,month,day,hour,weekday
0,126,23,2021-05-05 11:04:46.579,10,341,3.0,47.0,100,0.0,2021-05-05 11:04:46.579,2021-05-05,2021,5,5,11,3
1,166,41,2021-05-06 10:45:02.448,196,341,3.0,47.0,2,0.0,2021-05-06 10:45:02.448,2021-05-06,2021,5,6,10,4
2,167,42,2021-05-06 10:45:04.850,196,341,3.0,47.0,2,0.0,2021-05-06 10:45:04.850,2021-05-06,2021,5,6,10,4


### Exploratory Data Analysis

 - We are going to ask a series of questions:
     - What are the busiest times of the year to make note of?
     - What kinds of goods are sold the most frequently?
     - What kinds of goods are sold at higher quantities?
     - What are the average quantities sold for each SKU?
     - Are any of the types of goods sold at higher rates during certain parts of the year?
     - Can we detect any automated purchasing?
     - How many total merchants is this distribution center (DC) servicing?
     - How frequent do these merchants purchase from this DC?
     - Were these merchants customers throughout the entire time these data were collected?
     - What did these merchants purchase? How much?
     
#### Busiest times of the year:

![Plot of order traffic](https://github.com/carsonslater/mentored_research2022/blob/f61cc476974b3be31b968c53a5c2cf8a208e96c9/DC_order_traffic)

#### What kinds of goods are sold the most frequently?

In [9]:
sub_cat_cnt = baskets.groupby('sub_cat_id').count().sort_values(by=['id'], ascending = False)
sub_cat_cnt.head(5)

,id,order_id,placed_at,merchant_id,sku_id,top_cat_id,qty,price,datetime,date,year,month,day,hour,weekday
sub_cat_id,,,,,,,,,,,,,,,
48.0,2789,2789,2789,2789,2789,2789,2789,2789,2789,2789,2789,2789,2789,2789,2789
37.0,2736,2736,2736,2736,2736,2736,2736,2736,2736,2736,2736,2736,2736,2736,2736
31.0,2522,2522,2522,2522,2522,2522,2522,2522,2522,2522,2522,2522,2522,2522,2522
57.0,1792,1792,1792,1792,1792,1792,1792,1792,1792,1792,1792,1792,1792,1792,1792
105.0,1625,1625,1625,1625,1625,1625,1625,1625,1625,1625,1625,1625,1625,1625,1625


Because the data is obfuscated, we do not know which `sub_cat_id` corresponds to which kind of SKU. What we do know is that the SKU subcategories near the top of the dataframe above are the ones that appeared most often in purchases.

#### What about for `top_cat_id` and `sku_id`?

In [10]:
top_cat_cnt = baskets.groupby('top_cat_id').count().sort_values(by=['id'], ascending = False)
top_cat_cnt.head(5)

,id,order_id,placed_at,merchant_id,sku_id,sub_cat_id,qty,price,datetime,date,year,month,day,hour,weekday
top_cat_id,,,,,,,,,,,,,,,
4.0,8100,8100,8100,8100,8100,8100,8100,8100,8100,8100,8100,8100,8100,8100,8100
15.0,3437,3437,3437,3437,3437,3437,3437,3437,3437,3437,3437,3437,3437,3437,3437
3.0,3349,3349,3349,3349,3349,3349,3349,3349,3349,3349,3349,3349,3349,3349,3349
9.0,3266,3266,3266,3266,3266,3266,3266,3266,3266,3266,3266,3266,3266,3266,3266
12.0,2747,2747,2747,2747,2747,2747,2747,2747,2747,2747,2747,2747,2747,2747,2747


In [11]:
top_cat_cnt = baskets.groupby('sku_id').count().sort_values(by=['id'], ascending = False)
top_cat_cnt.head(5)

,id,order_id,placed_at,merchant_id,top_cat_id,sub_cat_id,qty,price,datetime,date,year,month,day,hour,weekday
sku_id,,,,,,,,,,,,,,,
962,580,580,580,580,580,580,580,580,580,580,580,580,580,580,580
761,577,577,577,577,577,577,577,577,577,577,577,577,577,577,577
824,522,522,522,522,522,522,522,522,522,522,522,522,522,522,522
875,479,479,479,479,479,479,479,479,479,479,479,479,479,479,479
696,454,454,454,454,454,454,454,454,454,454,454,454,454,454,454


#### Average Quantity of SKU per purchase

In [20]:
avg_qty = baskets.groupby('sku_id').mean()
avg_qty

,id,order_id,merchant_id,top_cat_id,sub_cat_id,qty,price,year,month,day,hour,weekday
sku_id,,,,,,,,,,,,
5,2184.000000,356.0,349.000000,14.0,37.0,4.000000,238500.0,2021.0,6.0,17.000000,10.000000,4.0
6,12462.000000,1100.0,349.000000,20.0,67.0,1.000000,0.0,2021.0,8.0,5.000000,9.000000,4.0
8,14059.000000,1236.0,349.000000,15.0,94.0,3.000000,0.0,2021.0,8.0,11.000000,7.000000,3.0
10,2028.000000,344.0,349.000000,9.0,48.0,2.000000,96000.0,2021.0,6.0,15.000000,9.000000,2.0
14,2202.000000,356.0,349.000000,9.0,48.0,1.000000,90500.0,2021.0,6.0,17.000000,10.000000,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2372,328974.666667,58974.0,767.333333,15.0,73.0,2.333333,149000.0,2022.0,7.0,8.333333,12.666667,3.0
2378,332776.000000,60179.0,1753.000000,4.0,57.0,3.000000,169000.0,2022.0,7.0,18.000000,14.000000,1.0
2381,331471.000000,59859.0,185.000000,12.0,12.0,1.000000,64500.0,2022.0,7.0,13.000000,22.000000,3.0
